# just some experimentation :))

In [1]:
# silence warnings
import warnings
warnings.filterwarnings("ignore")

# allow helper import
import sys
sys.path.append('../../assets/scripts')

from cheff import CheffAEModel, CheffLDM
from cxrage import load_model, age_fn
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.transforms.functional import to_tensor

In [2]:
device = 'cpu'
sdm_path = '../../assets/models/cheff_diff_uncond.pt'
ae_path = '../../assets/models/cheff_autoencoder.pt'

cheff_ldm = CheffLDM(model_path=sdm_path, device=device)
cheff_ae = CheffAEModel(model_path=ae_path, device=device)

# load cxr-age
cxr_age = load_model(path='../../assets/')
cxr_age.model.to(device)

# load, encode sample images
sample_imgs = [Image.open(f'../../assets/cxrs/nih/ae_nih{i}.png') for i in range(1, 6)]
with torch.no_grad():
    lreps = [cheff_ae.encode(to_tensor(img).unsqueeze(0).to(device)) for img in sample_imgs]

In [36]:
samples = cheff_ldm.sample(
    batch_size = 1,
    sampling_steps=100,
    eta=1.0,
    decode=False
)

Data shape for DDIM sampling is (1, 3, 64, 64), eta 1.0
Running DDIM Sampling with 100 timesteps


DDIM Sampler: 100%|██████████| 100/100 [02:41<00:00,  1.61s/it]


In [38]:

def spectral_normalization(latent, target_latent, num_components=10):
    """
    Adjust the spectral properties of the latent representation
    to match those of a target (realistic) latent.
    
    Args:
    - latent: The latent representation to adjust
    - target_latent: The target latent representation (assumed to be realistic)
    - num_components: Number of principal components to consider
    
    Returns:
    - The spectrally normalized latent representation
    """
    # Reshape latents to 2D
    b, c, h, w = latent.shape
    latent_2d = latent.reshape(b, c, -1)
    target_2d = target_latent.reshape(b, c, -1)
    
    # Compute SVD
    _, s, v = torch.svd(latent_2d, some=False)
    _, s_target, _ = torch.svd(target_2d, some=False)
    
    # Adjust singular values
    s_adjusted = s.clone()
    s_adjusted[:, :num_components] = s_target[:, :num_components]
    
    # Reconstruct the adjusted latent
    latent_adjusted = torch.matmul(torch.matmul(latent_2d, v), v.transpose(-2, -1))
    latent_adjusted = latent_adjusted * s_adjusted.unsqueeze(-1) / s.unsqueeze(-1)
    
    return latent_adjusted.reshape(b, c, h, w)

In [42]:
lrep = lreps[0].clone()

# spectral normalization
with torch.no_grad():
    lrep = spectral_normalization(lrep, samples[0])

# decode lrep
with torch.no_grad():
    n_img = cheff_ae.decode(lrep)

In [17]:
n_lrep = 0.7 * lreps[0][0] + 0.3 * samples[0]
n_img = cheff_ae.decode(n_lrep.unsqueeze(0))

In [43]:
# randomly sample latent space reps near lreps[0]
lrep = lreps[0]
lrep += torch.randn_like(lrep) * 0.5

img = n_img.squeeze().cpu().detach().numpy()

# display sample_imgs[0], img
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(sample_imgs[0])
ax[0].axis('off')
ax[0].set_title('Original Image')
img = ((img - img.min()) / (img.max() - img.min())).transpose(1, 2, 0)
ax[1].imshow(img, cmap='gray')
ax[1].axis('off')
ax[1].set_title('Generated Image')
plt.show()